In [1]:
import numpy as np
import pandas as pd

import os
import requests
from  bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By

from time import sleep
import random

In [9]:
browser = webdriver.Chrome()

browser.get("https://nba.com/games")
browser.maximize_window()

# cookies
sleep(1)
browser.find_element(by=By.ID, value="onetrust-accept-btn-handler").click()

# Ir a la página de equipos
sleep(2)
browser.find_element(by=By.XPATH, value='//*[@id="nav-ul"]/li[7]/a').click()

In [14]:
html_equipos = browser.page_source
soup = BeautifulSoup(html_equipos, "html.parser")
urls_equipos_nba = [a['href'] for a in soup.find_all('a', class_='Anchor_anchor__cSc3P TeamFigureLink_teamFigureLink__uqnNO') if '/stats/team/' in a['href']]
print(urls_equipos_nba)

['/stats/team/1610612738', '/stats/team/1610612751', '/stats/team/1610612752', '/stats/team/1610612755', '/stats/team/1610612761', '/stats/team/1610612741', '/stats/team/1610612739', '/stats/team/1610612765', '/stats/team/1610612754', '/stats/team/1610612749', '/stats/team/1610612737', '/stats/team/1610612766', '/stats/team/1610612748', '/stats/team/1610612753', '/stats/team/1610612764', '/stats/team/1610612743', '/stats/team/1610612750', '/stats/team/1610612760', '/stats/team/1610612757', '/stats/team/1610612762', '/stats/team/1610612744', '/stats/team/1610612746', '/stats/team/1610612747', '/stats/team/1610612756', '/stats/team/1610612758', '/stats/team/1610612742', '/stats/team/1610612745', '/stats/team/1610612763', '/stats/team/1610612740', '/stats/team/1610612759']


In [17]:
import os
import requests
from bs4 import BeautifulSoup

url_base = "https://www.nba.com"
web_equipos_nba = [url_base + equipo_url for equipo_url in urls_equipos_nba]

# Función para extraer la URL del logo de un equipo
def extraer_logos_equipo_bs4(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            logo_elemento = soup.select_one(".TeamHeader_logoBlock__WjNZB img")
            id_equipo = url.split('/')[-1]
            url_logo = logo_elemento['src'] if logo_elemento else None
            return id_equipo, url_logo
    except Exception as e:
        print(f"Error al procesar {url}: {e}")
    return None, None

# Función para descargar el logo
def descargar_logo(id_equipo, url, ruta_carpeta):
    if url:
        try:
            response = requests.get(url)
            if response.status_code == 200:
                nombre_archivo = f"{id_equipo}.svg"  # Usar el ID del equipo para el nombre del archivo
                ruta_completa = os.path.join(ruta_carpeta, nombre_archivo)

                with open(ruta_completa, 'wb') as file:
                    file.write(response.content)
        except Exception as e:
            print(f"Error al descargar {url}: {e}")

# Carpeta para guardar los logos
ruta_carpeta_logos = "logos_equipos"
if not os.path.exists(ruta_carpeta_logos):
    os.makedirs(ruta_carpeta_logos)

# Proceso para descargar cada logo
for url_equipo in web_equipos_nba:
    id_equipo, url_logo = extraer_logos_equipo_bs4(url_equipo)
    if url_logo:
        descargar_logo(id_equipo, url_logo, ruta_carpeta_logos)